In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

from scipy.fftpack import fftn, fftfreq, fftshift
from scipy.stats import binned_statistic


In [ ]:
def plot_settings(figsize=(6,6)):
    ''' For nicer plots.
    '''
    font = {'size'   : 20, 'family':'STIXGeneral'}
    axislabelfontsize='large'
    matplotlib.rc('font', **font)
    matplotlib.mathtext.rcParams['legend.fontsize']='small'
    matplotlib.rcParams['mathtext.fontset'] = 'cm'
    plt.rcParams['figure.figsize'] = figsize

def convert_format2D(arr, N):
    ret = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            ret[i,j] = arr[i + N * j]
    
    return ret

def read_snapshots2D(N, func=lambda p1, p2: np.arctan2(p1, p2)):
    ''' Note: Appends axion field to snapshots by default.
    '''
    snapshots = []
    count = 0
    done = False
    while not done:
        try:
            phi1 = convert_format2D(np.fromfile(f"snapshot{count}-phi1", dtype=np.float64), N)
            phi2 = convert_format2D(np.fromfile(f"snapshot{count}-phi2", dtype=np.float64), N)
            snapshots.append(func(phi1, phi2))
            count += 1
        except:
            done = True
    
    return snapshots

def convert_format3D(arr, N):
    ret = np.zeros((N, N, N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                ret[i,j,k] = arr[(i * N + j) * N + k]

    return ret

def read_snapshots3D(N, func=lambda p1, p2: np.arctan2(p1, p2)):
    ''' Note: Appends axion field to snapshots by default.
    '''
    snapshots = []
    count = 0
    done = False
    while not done:
        try:
            phi1 = convert_format3D(np.fromfile(f"snapshot{count}-phi1", dtype=np.float64), N)
            phi2 = convert_format3D(np.fromfile(f"snapshot{count}-phi2", dtype=np.float64), N)
            snapshots.append(func(phi1, phi2))
            count += 1
        except:
            done = True
    
    return snapshots


In [ ]:
# Show 2D snapshots:
%matplotlib inline
plot_settings(figsize=(13,12))

snapshots = read_snapshots2D(256)

for i in range(len(snapshots)):
    plt.imshow(snapshots[i], cmap=matplotlib.cm.twilight, interpolation='none')
    plt.xlabel(r"Comoving distance $x/f_a$")
    cbar = plt.colorbar(fraction=0.05, pad=0.04, shrink=0.75)
    cbar.set_ticks([np.pi-0.001, np.pi/2, 0, -np.pi/2, -np.pi+0.001])
    cbar.set_ticklabels([r"$\pi$", r"$\pi/2$","0",r"$-\pi/2$", r"$-\pi$"])
    cbar.set_label(r"$a(x)/f_a$", rotation=0, labelpad=20)
    plt.show()


In [ ]:
# Show 3D snapshots as 2D slices:
%matplotlib inline
plot_settings(figsize=(13,12))

snapshots = read_snapshots3D(128)

for i in range(len(snapshots)):
#     plt.imshow(snapshots[i][128//2], cmap=matplotlib.cm.viridis, interpolation='none')
#     plt.colorbar()
    plt.imshow(snapshots[i][128//2], cmap=matplotlib.cm.twilight, interpolation='none')
    plt.xlabel(r"Comoving distance $x/f_a$")
    cbar = plt.colorbar(fraction=0.05, pad=0.04, shrink=0.75)
    cbar.set_ticks([np.pi-0.001, np.pi/2, 0, -np.pi/2, -np.pi+0.001])
    cbar.set_ticklabels([r"$\pi$", r"$\pi/2$","0",r"$-\pi/2$", r"$-\pi$"])
    cbar.set_label(r"$a(x)/f_a$", rotation=0, labelpad=20)
    plt.show()


In [ ]:
# Read single snapshot:
%matplotlib inline
plot_settings(figsize=(13,12))

p1 = convert_format3D(np.fromfile("file-name", dtype=np.float32), 256)
p2 = convert_format3D(np.fromfile("file-name", dtype=np.float32), 256)

ax = np.arctan2(p1,p2)
sx = np.sqrt(p1**2 + p2**2)

plt.imshow(ax[80], cmap=matplotlib.cm.twilight, interpolation='none')
plt.title(r"Axion field $a(x)/f_a$")
locx, _ = plt.xticks([])
locy, _ = plt.yticks([])
cbar = plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_ticks([np.pi-0.001, np.pi/2, 0, -np.pi/2, -np.pi+0.001])
cbar.set_ticklabels([r"$\pi$", r"$\pi/2$","0",r"$-\pi/2$", r"$-\pi$"])
plt.show()

In [ ]:
# Time-series observables:
%matplotlib inline
plot_settings()

ret = pd.read_csv("time-series.csv")

t = np.array(ret['time'].tolist())
xi = np.array(ret['xi'].tolist())
# phi1_bar = np.array(ret['phi1_bar'].tolist())
# phi2_bar = np.array(ret['phi2_bar'].tolist())
# phidot1_bar = np.array(ret['phidot1_bar'].tolist())
# phidot2_bar = np.array(ret['phidot2_bar'].tolist())
# axion_bar = np.arctan2(phi1_bar, phi2_bar)
# saxion_bar = np.sqrt(phi1_bar**2 + phi2_bar**2)

plt.plot(t, xi, "x")
plt.xlabel(r"$t/f_a$")
plt.ylabel(r"$\xi$", rotation=0, labelpad=20)
plt.show()

# plt.scatter(phi1_bar, phi2_bar, marker=".", c=t, cmap=matplotlib.cm.viridis)
# plt.show()

# plt.plot(t, phi1_bar, "x", label="phi1")
# plt.plot(t, phi2_bar, "x", label="phi2")
# plt.plot(t, phidot1_bar, "x", label="phidot1")
# plt.plot(t, phidot2_bar, "x", label="phidot1")
# plt.legend()
# plt.show()

# plt.scatter(phi1_bar, phidot1_bar, marker=".", c=t, cmap=matplotlib.cm.viridis, label="phi1")
# plt.scatter(phi2_bar, phidot2_bar, marker="x", c=t, cmap=matplotlib.cm.viridis, label="phi2")
# plt.legend()
# plt.show()

In [ ]:
# Plot string snapshot:
%matplotlib notebook
plot_settings(figsize=(8,8))

from mpl_toolkits.mplot3d import Axes3D

ret = pd.read_csv("string-pos-file-name", header=None)
xs = ret[0].tolist()
ys = ret[1].tolist()
zs = ret[2].tolist()

ax = plt.axes(projection='3d')
ax.scatter3D(xs, ys, zs, c=zs, cmap=matplotlib.cm.viridis);
plt.show()

In [ ]:
# Power spectrum sanity check:
%matplotlib inline
plot_settings()

def pkphi(k, T_initial=4.0, lambdaPRS=1.0):
    m_eff_squared = lambdaPRS * (T_initial**2 / 3 - 1)
    omega = np.sqrt(k**2 + m_eff_squared);
    bose = 1 / (np.exp(omega / T_initial) - 1);
    return bose / omega

def power_spectrum2D(f,N,L):
    y = fftn(f)
    P = np.abs(y)**2
    kfreq = fftfreq(N)*N

    kfreq2D = np.meshgrid(kfreq, kfreq)
    K = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)
    K = K.flatten()
    P = P.flatten()
    kbins = np.arange(0.5, N//2, 1.)
    kvals = 0.5 * (kbins[1:] + kbins[:-1])
    # Pk, a, b = binned_statistic(K, P, statistic="mean", bins=kbins)
    Pk, bin_edges, _ = binned_statistic(K, P, statistic = "sum", bins = kbins)
    for i in range(len(Pk)):
        avg_bin = 0.5 * (bin_edges[i] + bin_edges[i+1])
        Pk[i] /= 2*np.pi*avg_bin
    # Pk *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)

    return kvals*2*np.pi/L,Pk

def power_spectrum3D(f,N,L):
    y = fftn(f)
    P = np.abs(y)**2
    kfreq = fftfreq(N)*N

    kfreq3D = np.meshgrid(kfreq, kfreq, kfreq)
    K = np.sqrt(kfreq3D[0]**2 + kfreq3D[1]**2 + kfreq3D[2]**2) 
    K = K.flatten()
    P = P.flatten()
    kbins = np.arange(0.5, N//2, 1.)
    kvals = 0.5 * (kbins[1:] + kbins[:-1])
    # Pk, a, b = binned_statistic(K, P, statistic = "mean", bins = kbins)
    Pk, bin_edges, _ = binned_statistic(K, P, statistic = "sum", bins = kbins)
    for i in range(len(Pk)):
        avg_bin = 0.5 * (bin_edges[i] + bin_edges[i+1])
        Pk[i] /= 4*np.pi*avg_bin**2
    # Pk *= 4/3 * np.pi * (kbins[1:]**3 - kbins[:-1]**3)

    return kvals*(2*np.pi/L),Pk

N = 128
L = N * 1.0
phi = convert_format3D(np.fromfile("phi1-initial", dtype=np.float64), N)

ks, pks = power_spectrum3D(phi, N, L)
plt.loglog(ks, pks, ".")
plt.loglog(ks, pkphi(ks, lambdaPRS=0.0)*(2*np.pi/N)**3)
plt.show()

# plt.loglog(ks, 1/ks**4/(N)**3/pks, ".-", label="4")
# plt.loglog(ks, 1/ks**3/(N)**3/pks, ".-", label="3")
# plt.loglog(ks, 1/ks**2/(N)**3/pks, ".-", label="2")
# plt.loglog(ks, 1/ks**1/(N)**3/pks, ".-", label="1")
# plt.legend()
# plt.show()

In [ ]:
# Test gaussianity with no interactions or hubble friction:
%matplotlib inline
plot_settings()

N = 256
L = N * 1.0
snapshots = read_snapshots2D(N, func=lambda p1, p2: p1) # Just pick out phi1 field for now.

for i in range(len(snapshots)):
    ks, pks = power_spectrum2D(snapshots[i], N, L)
    plt.loglog(ks, pks, ".")
    plt.loglog(ks, pkphi(ks, lambdaPRS=1.0)*(2*np.pi/L)**2)
    plt.show()